#### DEMO Import Generated Notebook

In [0]:
# sample notebook source, e.g., generated code
notebook_source="""
-- Databricks notebook source
--demo command import 1
select 'hello world from command 1'

-- COMMAND ----------

--demo command import 2
select 'hello world from command 2'

-- COMMAND ----------
"""

In [0]:
# encode the notebook: base64
import base64
encoded_notebook=base64.b64encode(bytes(notebook_source, 'utf-8'))

In [0]:
# sample notebook config 
# path = workspace target path
# 
notebook_config={
    "path": "/DEV/IMPORTABLE/new_generated_sql", 
    "content": encoded_notebook.decode(),
    "language": "SQL", 
    "overwrite": True, 
    "format": "SOURCE" 
}

In [0]:
# Workspace API to Import Notebook
import requests
import json

# undocumented dbutils apis, which may be evolving.
workspaceUrl = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
auth_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
            
def CreateNotebook(workspaceUrl: str, auth_token: str, notebook_config: dict) -> dict:
    api_url = f"{workspaceUrl}/api/2.0/workspace/import"
    headers = {"Authorization":"Bearer {}".format(auth_token), "Accept":"application/json" }
    payload = notebook_config
    results = requests.post(
        api_url,
        headers=headers,
        data=json.dumps(payload)
    )
    # write to event log for audit irl
    output={"action": {"import": notebook_config['path']},"api_status":results.status_code,"api_result":results.text}    
    return (output)

In [0]:
#create notebook on target workspace
CreateNotebook(workspaceUrl=workspaceUrl, auth_token=auth_token, notebook_config=notebook_config)